In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import newton

In [7]:
def GeneratePathsGBM(NoOfPaths, NoOfSteps, t1, T1, T, sigma, lambda1, frate, corp, K, numeraire):
    #finding the 'dt' value the limits of integration are already fixed i.e., from s to T i.e., time to maturity
    dt = t1 / float(NoOfSteps)
    if T/dt != int(T/dt) or t1/dt != int(t1/dt) or T1/dt != int(T1/dt):
      print("enter NoofSteps such that T and t1 are exact multiples of dt")
    time = np.arange(0,T+dt,dt)
    global NoOfSteps1
    NoOfSteps1 = NoOfSteps
    #generating standard normal distribution values for  'dW'
    Z = np.random.normal(0.0, 1.0, [NoOfPaths,len(time)])
    #creating vector for storing the interest rate values
    r = np.zeros([NoOfPaths, len(time) ])

    r[:, 0] = frate

    for i in range(0, len(time)-1):
      # Making sure that samples from a normal have mean 0 and variance 1
      if NoOfPaths > 1:
          Z[:, i] = (Z[:, i] - np.mean(Z[:, i])) / np.std(Z[:, i])

      #finding the value of nut for each time interval
      nut = 0 + lambda1 * frate + (np.power(sigma,2) / (2 * lambda1)) * (1 - np.exp(-2 * lambda1 * time[i]))

      #finding value of interest rate at every time step
      if numeraire != "tforward":
        r[:,i+1] = r[:,i] + (nut - lambda1 * r[:,i]) * dt + sigma * np.power(dt,0.5) * Z[:, i]
      else:
        r[:,i+1] = r[:,i] + (nut - lambda1 * r[:,i] - np.power(sigma,2) * b(time[i])) * dt + sigma * np.power(dt,0.5) * Z[:, i]

    #defining functions for the exact bond price formula
    def fb(t,T):
      return (1/lambda1) * (1 - np.exp(-lambda1*(T - t)))

    def fa(t,T):
      return (np.exp(-1*frate*T)/np.exp(-1*frate*t)) * (np.exp(fb(t,T) * (frate) - (np.power(sigma,2)/(4 * lambda1)) * (1 - np.exp(-2 * lambda1 * t)) * np.power(fb(t,T),2)))

    def P(t,T,rt):
      return (fa(t,T) * np.exp(-1 * fb(t,T) * rt))

    value=np.zeros([NoOfPaths,len(time)-NoOfSteps])

    for i in range(NoOfSteps,len(time)):
      value[:,i-NoOfSteps] = fa(time[i],T)*np.exp(-fb(time[i],T)*frate)

    disv = value[:,0] * np.exp(-1 * np.sum(r[:,0:NoOfSteps] * dt))
    #finding the value of the option at T1
    opt = np.maximum(value[:,int(T1/dt - NoOfSteps)] - K,0)
    #discounting the option value to time "t1" over simulated paths of interest rates
    disopt = opt * np.exp(-1 * np.sum(r[:,NoOfSteps:int(T1/dt)]*dt,axis=1))
    #defining the zero bond coupon exact formula
    def zbc(t,T1,T,K, rt):
      sigmap = sigma * np.power(((1 - np.exp(-2* lambda1 * (T1 - t)))/(2*lambda1)),0.5) * fb(T1,T)
      h = (1/sigmap)*(np.log(P(t,T,rt)/(P(t,T1,rt)*K))) + (sigmap/2)
      if sigmap==0:
        h=float('inf')
      return P(t,T, rt) * norm.cdf(h) - K * P(t,T1, rt) * norm.cdf(h - sigmap)

    #option value at t1 found using the exact formula for zero bond call option
    optform = zbc(time[NoOfSteps],T1, T, K, frate)
    #option value at present time t=0 found using exact formula for zero bond call option
    optform0 = zbc(time[0],T1, T, K, frate)
    #option value at t1 discounted to present time t=0 over simulated interest rate paths.
    optformdis = optform * np.exp(-1 * np.sum(r[:,0:NoOfSteps] * dt,axis = 1))

    return {"disv":disv,"value":value,"dt":dt,"time":time,"opt":opt,"disopt":disopt,"optform":optform, "optformdis":optformdis, "optform0":optform0}

In [8]:
k=GeneratePathsGBM(10, 5000, 5, 7, 10, 0.01, 1, 0.1, "c", 0.5, "moneybank")

In [9]:
#this is the array containing prices of options at T1 discounted to t1
k["disopt"]

array([0.19811534, 0.19632134, 0.19730642, 0.19794303, 0.19829626,
       0.19543592, 0.19751107, 0.19773574, 0.19535056, 0.19733035])

In [10]:
#this is the array containing prices of options at t1 found using the formula
k["optform"]

0.1971579755022278

In [11]:
#this is the value of the option at time 0
k["optform0"]

0.1195867892757376

In [12]:
#this is the array containing values of the option found from discounting the value of options at 't' to '0'
k["optformdis"]

array([0.12683792, 0.1165727 , 0.12012399, 0.11784319, 0.12278689,
       0.11691255, 0.11890861, 0.12014705, 0.11930967, 0.11655016])